In [ ]:
import gurobipy as gp
from gurobipy import GRB 
import openpyxl
from openpyxl import load_workbook

### Cell 1: Preprocessing (input data from excel and convert of currency unit)
wb = load_workbook('Ex4.xlsx')
sheet1 = wb['Sheet1']

rate = 1.2

I = set(range(1,sheet1['A6'].value + 1)) # set of facility
J = set(range(1,sheet1['F2'].value + 1)) # set of customer
b = {j: sheet1.cell(row=7, column=c).value for c in range(2,7) for j in J if j == sheet1.cell(row=2, column=c).value} # demand of customer
a = {i: sheet1.cell(row=r, column=7).value for r in range(3,7) for i in I if i == sheet1.cell(row=r, column=1).value} # capacity of facility
c = {i: {j: sheet1.cell(row=r, column=c).value * rate 
         for c in range(2,7) for j in J if j == sheet1.cell(row=2, column=c).value} 
         for r in range(3,7) for i in I if i == sheet1.cell(row=r, column=1).value} # transportation cost from i to j

In [2]:
### Cell 2: Implementation
model = gp.Model("Transportation Problem")
x = model.addVars(((i,j) for i in I for j in J), lb=0, vtype = GRB.CONTINUOUS, name = "quantity")
model.setObjective(gp.quicksum(x[i,j] * c[i][j] for i in I for j in J), GRB.MINIMIZE)
model.addConstrs((gp.quicksum(x[i,j] for j in J) <= a[i] for i in I), name="capacity constraints")
model.addConstrs((gp.quicksum(x[i,j] for i in I) == b[j] for j in J), name="demand constraints")
model.optimize()

Set parameter Username
Set parameter LicenseID to value 2656700
Academic license - for non-commercial use only - expires 2026-04-25
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Linux Mint 22.1")

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 20 columns and 40 nonzeros
Model fingerprint: 0x267ac783
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+01]
Presolve time: 0.01s
Presolved: 9 rows, 20 columns, 40 nonzeros



Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8200000e+02   4.700000e+01   0.000000e+00      0s
       4    4.2720000e+02   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.03 seconds (0.00 work units)
Optimal objective  4.272000000e+02


In [3]:
### Cell 3: Preprocessing (write in the optimal solution)
for j in J:
    for i in I:
        sheet1.cell(row=11+i, column=1+j, value=round(x[i,j].X,0))
sheet1['C17'].value = model.objVal
wb.save('Ex4.xlsx')

In [4]:
### Cell 4: Preprocessing (result output)
for i in I: # calculate the cost for every i in I
    print(f'Cost for {i}: {round(sum(x[i,j].X * c[i][j] for j in J),1)}')
print(f'The total cost is {model.objVal} Euros.')

Cost for 1: 90.0
Cost for 2: 144.0
Cost for 3: 79.2
Cost for 4: 114.0
The total cost is 427.2 Euros.


In [ ]:
def transportation_problem(filename, rate):
    wb = load_workbook(filename)
    sheet1 = wb['Sheet1']
    I = set(range(1,sheet1['A6'].value + 1)) # set of facility
    J = set(range(1,sheet1['F2'].value + 1)) # set of customer
    b = {j: sheet1.cell(row=7, column=c).value for c in range(2,7) for j in J if j == sheet1.cell(row=2, column=c).value} # demand of customer
    a = {i: sheet1.cell(row=r, column=7).value for r in range(3,7) for i in I if i == sheet1.cell(row=r, column=1).value} # capacity of facility
    c = {i: {j: sheet1.cell(row=r, column=c).value * rate 
             for c in range(2,7) for j in J if j == sheet1.cell(row=2, column=c).value} 
             for r in range(3,7) for i in I if i == sheet1.cell(row=r, column=1).value} # transportation cost from i to j
    # Gurobi Model
    model = gp.Model("Transportation Problem")
    x = model.addVars(((i,j) for i in I for j in J), lb=0, vtype = GRB.CONTINUOUS, name = "quantity")
    model.setObjective(gp.quicksum(x[i,j] * c[i][j] for i in I for j in J), GRB.MINIMIZE)
    model.addConstrs((gp.quicksum(x[i,j] for j in J) <= a[i] for i in I), name="capacity constraints")
    model.addConstrs((gp.quicksum(x[i,j] for i in I) == b[j] for j in J), name="demand constraints")
    model.setParam('OutputFlag', 0)
    model.optimize()
    # Result output
    if model.status == GRB.OPTIMAL:
        print(f'\nThe total cost of instance (rate={rate}) is {model.objVal} Euros.')
        for i in I:
            print(f'Cost for {i}: {round(sum(x[i,j].X * c[i][j] for j in J),1)}')
        else:
            print(f"Instance (rate={rate}) has no optimal solution.")

In [6]:
rate_set = {1, 2.18, 1.35, 1.79, 2}
for rate in rate_set:
    transportation_problem('Ex4.xlsx',rate)


The total cost of instance (rate=1.79) is 637.24 Euros.
Cost for 1: 134.2
Cost for 2: 171.8
Cost for 3: 107.4
Cost for 4: 223.8
Instance (rate=1.79) has no optimal solution.

The total cost of instance (rate=2.18) is 776.0800000000002 Euros.
Cost for 1: 163.5
Cost for 2: 209.3
Cost for 3: 130.8
Cost for 4: 272.5
Instance (rate=2.18) has no optimal solution.

The total cost of instance (rate=2) is 712.0 Euros.
Cost for 1: 150.0
Cost for 2: 240.0
Cost for 3: 132.0
Cost for 4: 190.0
Instance (rate=2) has no optimal solution.

The total cost of instance (rate=1.35) is 480.6 Euros.
Cost for 1: 101.3
Cost for 2: 129.6
Cost for 3: 81.0
Cost for 4: 168.8
Instance (rate=1.35) has no optimal solution.

The total cost of instance (rate=1) is 356.0 Euros.
Cost for 1: 75.0
Cost for 2: 120.0
Cost for 3: 66.0
Cost for 4: 95.0
Instance (rate=1) has no optimal solution.
